In [16]:
import os
import glob
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import operator
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
os.chdir("/Users/Ruihao/Desktop/course/AML/Final/data")

train_featfc, test_featfc = [], []
for line in open("./features_train/features_resnet1000_train.csv"):
    l = line.strip().split(",")
    train_featfc.append(l)
train_featfc = np.array(train_featfc)
train_featfc = np.delete(train_featfc, 0, axis=1)
train_featfc = np.array(train_featfc, dtype = np.float32)

for line in open("./features_test/features_resnet1000_test.csv"):
    l = line.strip().split(",")
    test_featfc.append(l)
test_featfc = np.array(test_featfc)
test_featfc = np.delete(test_featfc, 0, axis=1)
test_featfc = np.array(test_featfc, dtype = np.float32)


train_featpool, test_featpool=[],[]
for line in open("./features_train/features_resnet1000intermediate_train.csv"):
    l = line.strip().split(",")
    train_featpool.append(l)
train_featpool=np.array(train_featpool)
train_featpool = np.delete(train_featpool, 0, axis=1)
train_featpool = np.array(train_featpool, dtype = np.float32)

for line in open("./features_test/features_resnet1000intermediate_test.csv"):
    l=line.strip().split(",")
    test_featpool.append(l)
test_featpool = np.array(test_featpool)
test_featpool = np.delete(test_featpool, 0, axis=1)
test_featpool = np.array(test_featpool, dtype = np.float32)


print (test_featfc.shape)
print (train_featfc.shape)
print (test_featpool.shape)
print (train_featpool.shape)


(2000, 1000)
(10000, 1000)
(2000, 2048)
(10000, 2048)


In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
total_tag_dict = list()
def get_tag_dict(path):
    # open all description trianing file(5 sentences txt)
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            sentence = content.split("\n")
            for k in sentence:
                sentence2 = k.split(":")
                for str in sentence2:
                    if str not in total_tag_dict and str != "":
                        total_tag_dict.append(str)
get_tag_dict("./tags_train/")
print len(total_tag_dict)
print total_tag_dict

91
['indoor', 'teddy bear', 'animal', 'cat', 'electronic', 'cell phone', 'person', 'accessory', 'handbag', 'furniture', 'dining table', 'kitchen', 'knife', 'food', 'carrot', 'book', 'wine glass', 'fork', 'cake', 'chair', 'pizza', 'umbrella', 'outdoor', 'stop sign', 'sports', 'kite', 'giraffe', 'potted plant', 'appliance', 'refrigerator', 'bowl', 'apple', 'orange', 'microwave', 'oven', 'toaster', 'spoon', 'sink', 'vase', 'vehicle', 'train', 'motorcycle', 'bottle', 'bed', 'boat', 'tv', 'remote', 'zebra', 'tennis racket', 'skis', 'suitcase', 'backpack', 'snowboard', 'dog', 'banana', 'keyboard', 'mouse', 'sheep', 'broccoli', 'scissors', 'cup', 'tie', 'airplane', 'bear', 'laptop', 'clock', 'elephant', 'car', 'skateboard', 'bird', 'bench', 'surfboard', 'horse', 'truck', 'bus', 'traffic light', 'baseball bat', 'baseball glove', 'couch', 'cow', 'bicycle', 'sports ball', 'fire hydrant', 'sandwich', 'hot dog', 'toothbrush', 'donut', 'toilet', 'parking meter', 'frisbee', 'hair drier']


In [3]:
tag_dict = {} # HashMap (word, id), word stands for high frequency word while id is assigned to it
index = 0
for w in total_tag_dict:
    tag_dict[w] = index
    index += 1 
print len(tag_dict)

91


In [4]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
total_word_dict = {}
def get_word_dict(path):
    # open all description trianing file(5 sentences txt)
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            # Lemmatizing & Remove Stop Words
            sentence = filter(None, re.split(r'[^a-zA-Z\']', content.lower()))
            stop = set(stopwords.words('english'))
            #print stop
            lmtzr = WordNetLemmatizer()
            parsed = [lmtzr.lemmatize(word) for word in sentence if word not in stop]
            #print parsed
            for word in parsed:
                #w = stemmer.stem(word)
                if word in total_word_dict:
                    total_word_dict[word] += 1
                else:
                    total_word_dict[word] = 1
# get the word dictionary for training description
# HashMap: (word, frequency)
get_word_dict("./descriptions_train/")

print total_word_dict

{'raining': 1, 'childern': 1, 'yellow': 737, 'four': 346, 'hanging': 314, 'woody': 1, 'comically': 1, 'marching': 2, 'snowing': 2, 'sunlit': 3, 'deckered': 5, 'flicking': 1, u'shaving': 2, 'sinking': 1, 'swivel': 1, 'deli': 6, 'regional': 1, 'leisurely': 2, 'politician': 2, 'stabbed': 1, 'bringing': 1, 'wooded': 54, 'prize': 2, 'wooden': 697, 'satchel': 3, 'frizbee': 5, 'crotch': 2, 'cheeto': 1, 'sailboarder': 1, 'ornamental': 6, 'charter': 2, 'daybed': 1, 'tired': 6, 'ornate': 38, 'bacon': 39, 'pulse': 1, 'elegant': 10, 'rusty': 22, 'tether': 1, 'snuggled': 2, 'theory': 2, 'relieving': 1, u'centimeter': 1, 'cooking': 89, u'numeral': 22, u'crouch': 9, 'reporter': 1, u'herb': 8, 'error': 2, 'herd': 290, 'china': 12, 'dogwood': 1, 'dorm': 3, 'strewn': 3, 'k': 1, 'climbed': 1, u'outskirt': 1, 'military': 47, "i'm": 1, 'climber': 1, 'toowoomba': 1, 'golden': 26, 'projection': 1, 'disability': 1, 'brought': 2, u'remnant': 1, 'piecces': 1, 'stern': 2, u'unit': 14, 'hoping': 1, 'spoke': 1, 'g

In [5]:
word_dict = {} # HashMap (word, id), word stands for high frequency word while id is assigned to it
index = 0
# for all words in dictionary, if frequency > threshold, identify it as a high frequency word 
# assign an index to it, and increase the index
for w in total_word_dict:
    if total_word_dict[w] > 10:
        word_dict[w] = index
        index += 1 
print word_dict
print len(word_dict)

{'yellow': 0, 'four': 1, 'sleep': 1351, 'asian': 887, 'ornate': 5, 'trolley': 1352, 'feeding': 1353, 'chinese': 1587, u'swan': 888, 'bike': 440, u'blanket': 442, 'rise': 1354, 'vase': 889, 'closeup': 443, 'fireplace': 1355, 'wooded': 3, 'baking': 890, u'parrot': 140, 'wooden': 4, 'uncooked': 891, 'heading': 892, 'clothes': 893, 'enjoy': 1358, u'bicycle': 1359, 'quilt': 894, 'hanging': 2, 'bacon': 6, 'frosted': 1360, u'horn': 444, 'chef': 445, 'surrounding': 1361, 'rusty': 7, 'street': 1362, 'panda': 446, 'shining': 942, 'blue': 1363, 'sesame': 304, 'cooking': 8, u'numeral': 9, 'net': 896, 'consisting': 448, 'men': 897, 'herd': 10, 'bookcase': 449, 'china': 11, 'path': 1364, 'cardboard': 898, 'dry': 899, 'luggage': 1365, 'elaborate': 69, 'smoke': 450, 'military': 12, "someone's": 900, 'york': 1743, 'golden': 13, 'straw': 1366, 'overpass': 1488, 'sliced': 901, 'visible': 1367, u'unit': 14, 'plow': 451, 'army': 903, 'sweater': 452, 'watering': 904, 'overhead': 96, 'calm': 906, 'type': 907

In [6]:
total_dict = {}
index = 0
for w in word_dict:
    if w not in total_dict:
        total_dict[w] = index
        index = index + 1
for w in tag_dict:
    if w not in total_dict:
        total_dict[w] = index
        index = index + 1
        
# print len(total_dict)
# print (total_dict)

def process_des(path, thresh):
    #HashMap (filename, array)
    # filename example could be 1.txt etc. 
    # array should be a 1644 length, telling us for this specific file (ex. 1.txt)
    # the frequency of all high frequency word (for example, the word yellow appears 12 times in total)
    # but in the current file, it appears 3 times
    des_vec = {}
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            # Lemmatizing & Remove Stop Words
            cur = [0] * len(total_dict)
            sentence = filter(None, re.split(r'[^a-zA-Z\']', content.lower()))
            stop = set(stopwords.words('english'))
            #print stop
            lmtzr = WordNetLemmatizer()
            parsed = [lmtzr.lemmatize(word) for word in sentence if word not in stop]
            #print parsed
            for word in parsed:
                #w = stemmer.stem(word)
                if word in total_dict:
                    cur[total_dict[word]] = 1
            des_vec[filename.split('/')[-1]] = cur
    return des_vec # ex. {"0.txt": [1,0,0,2,0,0,0,0,.....]; "1.txt" : [2,1,1,1,0,0,1,....]}

thresh = 10
train_des = process_des("./descriptions_train/", thresh)
print train_des["0.txt"]
print len(train_des["0.txt"])
test_des = process_des("./descriptions_test/", thresh)
#print test_des["0.txt"]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [7]:
def process_des(path):
    des_vec = {}
    for filename in glob.glob(os.path.join(path, "*.txt")):
        with open(filename, "r") as content_file:
            content = content_file.read()
            sentence = content.split("\n")
            cur = [0] * len(total_tag_dict)
            for k in sentence:
                sentence2 = k.split(":")
                for str in sentence2:
                    #print str
                    if str in total_tag_dict:
                        cur[tag_dict[str]] = 1
            des_vec[filename.split('/')[-1]] = cur
    return des_vec 

train_tag_des = process_des("./tags_train/")
print train_tag_des["0.txt"]
test_tag_des = process_des("./tags_test/")
#print test_des["0.txt"]

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
Y = []
for k in train_tag_des:
    #train_tag_des[k].append(k)
    Y.append(train_tag_des[k])
Y = np.array(Y, dtype = np.float32)

Y = np.concatenate((Y, train_featfc), axis=1)
Y = np.concatenate((Y, train_featpool), axis=1)
print Y.shape

(10000, 3139)


In [9]:
# first part for SVM
X = []
for k in train_des:
    #train_des[k].append(k)
    X.append(train_des[k])
X = np.array(X, dtype = np.float32)
print X.shape

(10000, 1822)


In [13]:
from sklearn.neural_network import MLPRegressor
# train_x = total_bow[0:10000] # 这里是training的description
# train_y = vis # 这里是train的resNet
# 在这里加入模型
reg = MLPRegressor(solver='sgd', activation='logistic', shuffle=True)
reg.fit(X, Y)
p = reg.predict(test_des["0.txt"])
print p

[[ 0.13272808 -0.00929935  0.23407873 ...,  0.29468996  0.45775007
   0.34280579]]


/Users/Ruihao/anaconda/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/Ruihao/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [20]:
#print p.shape
X_test = []
for k in test_des:
    #train_des[k].append(k)
    X_test.append(test_des[k])
X_test = np.array(X_test, dtype = np.float32)
print X_test.shape

Y_test = []
for k in test_tag_des:
    #train_tag_des[k].append(k)
    #print k
    # k的value很重要 你看到k就知道这是哪一个txt，然后你根据k的value去hashmap里面找相关test_featpc和test_featpool的vector
    # 这个hashmap 你在一开始读入文件的时候就可以build一下  我把第0行给删了 其实第0行还是有些用处的 可以用来build hashmap
    Y_test.append(test_tag_des[k])
Y_test = np.array(Y_test, dtype = np.float32)

Y_test = np.concatenate((Y_test, test_featfc), axis=1)
Y_test = np.concatenate((Y_test, test_featpool), axis=1)
print Y_test.shape


(2000, 1822)
887.txt
979.txt
324.txt
857.txt
1485.txt
1244.txt
1290.txt
1498.txt
1988.txt
797.txt
1787.txt
1114.txt
762.txt
1714.txt
1756.txt
1041.txt
1522.txt
308.txt
1874.txt
938.txt
907.txt
1277.txt
540.txt
149.txt
10.txt
1833.txt
495.txt
1987.txt
1721.txt
956.txt
1078.txt
385.txt
256.txt
572.txt
105.txt
367.txt
1394.txt
373.txt
1562.txt
723.txt
1120.txt
950.txt
893.txt
1421.txt
1435.txt
1932.txt
36.txt
535.txt
1959.txt
210.txt
65.txt
301.txt
84.txt
1085.txt
1125.txt
992.txt
278.txt
594.txt
700.txt
1067.txt
1341.txt
919.txt
341.txt
456.txt
292.txt
1073.txt
1326.txt
31.txt
43.txt
1186.txt
1784.txt
668.txt
761.txt
738.txt
1990.txt
229.txt
1493.txt
879.txt
655.txt
1023.txt
394.txt
1506.txt
1764.txt
1250.txt
121.txt
1167.txt
1845.txt
589.txt
1409.txt
1011.txt
1655.txt
1408.txt
290.txt
176.txt
998.txt
719.txt
726.txt
1236.txt
1253.txt
470.txt
1670.txt
948.txt
1471.txt
1363.txt
538.txt
345.txt
920.txt
576.txt
1801.txt
777.txt
1212.txt
637.txt
1704.txt
1152.txt
6.txt
1865.txt
966.txt
501.t